# Default Setting

In [ ]:
# TODO: this chuck can be written as an initialization function in the script with enables to automatically detect the connection 
#   between the paired bluetooth devices and the PC
import sounddevice as sd

# # enable to detect bluetooth devices, if and only if the devices are paired
sd._terminate()
sd._initialize()
sd.default.reset() # reset all the default setting
fs = 24000
sd.default.samplerate = fs # specify sample rate
sd.default.channels = 1, 2
sd.default.dtype = 'float32', 'float32' # specify data type
sd.default.channels = 1, 2 # specify the input/output channels
# sd.default.device = input_device, ouput_device 
sd.default.latency
sd.default.extra_settings
sd.default.blocksize
sd.default.clip_off
sd.default.never_drop_input
sd.default.prime_output_buffers_using_stream_callback = False # fill initial buffers with zeros (silence)

In [ ]:
sd.query_devices()
sd.default.device = 0, 1

In [ ]:
import sounddevice as sd

duration = 10
t = sd.rec(int(duration * fs))

In [ ]:
t

In [ ]:
sd.get_status()

In [ ]:
sd.play(t)

In [ ]:
t2 = sd.playrec(t)

In [ ]:
sd.play(t2)

# Recording with Arbitrary Duration

In [ ]:
import sounddevice as sd
import numpy as np

sd.default.reset() # reset all the default setting
fs = 24000
sd.default.samplerate = fs # specify sample rate
sd.default.channels = 1, 2
sd.default.dtype = 'float32', 'float32' # specify data type
sd.default.channels = 1, 2 # specify the input/output channels

In [5]:
sd.query_devices()

> 0 External Microphone, Core Audio (1 in, 0 out)
< 1 External Headphones, Core Audio (0 in, 2 out)
  2 MacBook Pro Microphone, Core Audio (1 in, 0 out)
  3 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  4 ZoomAudioDevice, Core Audio (2 in, 2 out)

In [8]:
sd.query_devices()[0]

{'name': 'External Microphone',
 'hostapi': 0,
 'max_input_channels': 1,
 'max_output_channels': 0,
 'default_low_input_latency': 0.004625,
 'default_low_output_latency': 0.01,
 'default_high_input_latency': 0.013958333333333333,
 'default_high_output_latency': 0.1,
 'default_samplerate': 48000.0}

In [6]:
import sounddevice as sd
import numpy as np
import time
import queue

sd._terminate()
sd._initialize()
sd.default.reset()  # reset all the default setting
fs = 24000
sd.default.samplerate = fs  # specify sample rate
sd.default.channels = 1, 2
sd.default.dtype = 'float32', 'float32'  # specify data type
sd.default.channels = 1, 2  # specify the input/output channels

audio = np.zeros(shape=(24000, 1))
q = queue.Queue()


def callback(in_data, frames, time, status):
    """This is called (from a separate thread) for each audio block."""
    q.put(in_data.copy())

try:
    with sd.InputStream(samplerate=24000,
                        device='External Microphone',
                        dtype='float32',
                        channels=1,
                        callback=callback):
        start_time = time.time()

        print('#' * 80)
        print('press Ctrl+C to stop the recording')
        print('#' * 80)
        while True:
            audio = np.append(audio, q.get(), axis=0)
except KeyboardInterrupt:
    end_time = time.time()
    print('\nRecording finished: costs {} {}'.format(end_time - start_time,
                                                     np.shape(audio)))
    start_time = time.time()
    sd.play(audio)
    sd.wait()
    end_time = time.time()
    print('\nPlaying finished: costs {}'.format(end_time - start_time))

################################################################################
press Ctrl+C to stop the recording
################################################################################

Recording finished: costs 4.677999973297119 (133032, 1)

Playing finished: costs 5.716684103012085


In [ ]:
np.shape(audio)